In [1]:
import pandas as pd
import os
import csv
import re
import itertools

## Merging all transcripts .txt files

In [2]:
data = pd.read_csv('../raw_data/Transcript/Segmented/Combined/_0efYOjQYRc.txt', header = None, sep='///')

<ipython-input-2-271f3e62120d>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('../raw_data/Transcript/Segmented/Combined/_0efYOjQYRc.txt', header = None, sep='///')


In [3]:
directory = '../raw_data/Transcript/Segmented/Combined'

In [4]:
for filename in os.listdir(directory):
    new_data = pd.read_csv(f'../raw_data/Transcript/Segmented/Combined/{filename}', header = None, sep='///', quoting=csv.QUOTE_NONE) 
    data = pd.concat([data, new_data])

<ipython-input-4-cc9c9b79fd80>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  new_data = pd.read_csv(f'../raw_data/Transcript/Segmented/Combined/{filename}', header = None, sep='///', quoting=csv.QUOTE_NONE)


In [5]:
data = data.rename(columns={0 : 'all_data'})

In [6]:
data['ID'] = data['all_data'].apply(lambda x: x.split('__')[0].strip('_'))
data['clip'] = data['all_data'].apply(lambda x: x.split('__')[1].strip('_'))
data['Phrase'] = data['all_data'].apply(lambda x: x.split('__')[-1].strip('_'))

In [7]:
data = data.drop(columns='all_data')

In [8]:
data['ID'] = data['ID'].astype(str)
data['clip'] = data['clip'].astype(str)
data["Segment_ID"] = data["ID"] + data["clip"]

In [9]:
len(data)

44969

## Removing ~40 audio files

In [10]:
remove = pd.read_csv('../raw_data/remove.csv', header = None)

In [11]:
data = data.drop(labels = remove)

## Merging with y -labels

In [12]:
labels = pd.read_csv('../backinthessr/data/y1_sentiment.csv')

In [13]:
labels['Segment_ID'] = labels['Segment_ID'].apply(lambda x: x.strip('sqa_mosi/'))
labels['Segment_ID'] = labels['Segment_ID'].apply(lambda x: x.strip('pom_extra/'))

In [14]:
len(set(data.Segment_ID).intersection(set(labels.Segment_ID)))

18637

In [15]:
final_data = pd.merge(data, labels, how='inner', on='Segment_ID')
final_data = final_data.drop_duplicates()

In [16]:
final_data

,ID,clip,Phrase,Segment_ID,Answer.sentiment
0,0efYOjQYRc,3,He is the co-founder of Rossen and Vettese Lim...,0efYOjQYRc3,0.333333
2,0efYOjQYRc,4,Chairman Glenn Ives said that Deloitte is very...,0efYOjQYRc4,0.666667
4,-wMB_hJL-3o,4,The best way to help a depressed person is to ...,-wMB_hJL-3o4,1.333333
5,-wMB_hJL-3o,6,There are many different treatments helpful fo...,-wMB_hJL-3o6,0.666667
6,-wMB_hJL-3o,7,So if you have a loved one who is depressed th...,-wMB_hJL-3o7,0.333333
...,...,...,...,...,...
18634,2h9VVQUZjK0,8,"In the digital age, though, technology has ma...",2h9VVQUZjK08,0.000000
18635,2h9VVQUZjK0,9,You might sometimes still see a corporate sea...,2h9VVQUZjK09,1.333333
18636,2h9VVQUZjK0,10,"Today, state laws recognize the signature of ...",2h9VVQUZjK010,1.000000
18637,2h9VVQUZjK0,11,"Originally, official stock certificates serve...",2h9VVQUZjK011,0.000000


## Merging with audio files

In [17]:
audio = pd.read_csv('../raw_data/files.csv', header = None)

In [18]:
audio = audio.rename(columns={0 : 'Segment_ID'})

In [19]:
audio['Segment_ID'] = audio['Segment_ID'].apply(lambda x: x.strip('.wav'))

In [20]:
audio['Segment_ID'] = audio['Segment_ID'].apply(lambda x: x.replace('_', ''))

In [21]:
len(set(final_data.Segment_ID).intersection(set(audio.Segment_ID)))

13553

In [22]:
final_data_2 = pd.merge(final_data, audio, how='inner', on='Segment_ID')
final_data_2 = final_data_2.drop_duplicates()

In [23]:
final_data_2

,ID,clip,Phrase,Segment_ID,Answer.sentiment
0,0efYOjQYRc,3,He is the co-founder of Rossen and Vettese Lim...,0efYOjQYRc3,0.333333
1,0efYOjQYRc,4,Chairman Glenn Ives said that Deloitte is very...,0efYOjQYRc4,0.666667
2,56276,1,I want to tell you a little bit about this (u...,562761,-0.666667
3,56276,2,Now I will you that this is probably one of t...,562762,-3.000000
4,56276,3,"This is one of the most stupidest movies, (um...",562763,-3.000000
...,...,...,...,...,...
13548,2h9VVQUZjK0,8,"In the digital age, though, technology has ma...",2h9VVQUZjK08,0.000000
13549,2h9VVQUZjK0,9,You might sometimes still see a corporate sea...,2h9VVQUZjK09,1.333333
13550,2h9VVQUZjK0,10,"Today, state laws recognize the signature of ...",2h9VVQUZjK010,1.000000
13551,2h9VVQUZjK0,11,"Originally, official stock certificates serve...",2h9VVQUZjK011,0.000000


## Splitting into train & test sets

In [24]:
from sklearn.model_selection import train_test_split

In [34]:
X = final_data_2[['Segment_ID', 'Phrase', 'clip', 'ID']]
y = final_data_2[['Answer.sentiment', 'Segment_ID']]

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [43]:
X_train.to_csv('X_train', index=False)

In [44]:
y_train.to_csv('y_train', index=False)

In [45]:
X_test.to_csv('X_test', index=False)

In [46]:
y_test.to_csv('y_test', index=False)